In [ ]:
example = """
029A
980A
179A
456A
379A
""".strip().splitlines()

In [ ]:
from functools import partial

numpad = {
    "0": (3, 1),
    "1": (2, 0),
    "2": (2, 1),
    "3": (2, 2),
    "4": (1, 0),
    "5": (1, 1),
    "6": (1, 2),
    "7": (0, 0),
    "8": (0, 1),
    "9": (0, 2),
    "A": (3, 2),
}

dirpad = {
    "^": (0, 1),
    "A": (0, 2),
    "<": (1, 0),
    "v": (1, 1),
    ">": (1, 2),
}

def steps(a, b, pad):
    fy, fx = pad[a]
    ty, tx = pad[b]
    oy, ox = fy-ty, fx-tx
    res = {}
    if oy > 0:
        res["^"] = oy
    else:
        res["v"] = -oy
    if ox > 0:
        res["<"] = ox
    else:
        res[">"] = -ox
    return res

steps_numpad = partial(steps, pad=numpad)
steps_dirpad = partial(steps, pad=dirpad)

In [ ]:
from shapely import length


def steps2code(steps, order=0):
    key = list(steps)[order]
    otherkey = list(steps)[int(not order)]
    return key*steps[key] + otherkey*steps[otherkey] + "A"


chardir = {
    "^": (-1, 0),
    "v": (1, 0),
    ">": (0, 1),
    "<": (0, -1),
    "A": (0, 0),
}

def verif_code(code, start, pad, avoid, i):
    y, x = pad[start]
    if code == ">>^A":
        print(start, y, x, i)
    for e in code:
        y += chardir[e][0]
        x += chardir[e][1]
        if code == ">>^A":
            print(y, x, i)
        if (y, x) == avoid:
            return False
    return True

verif_numpad_code = partial(verif_code, pad=numpad, avoid=(3, 0))
verif_dirpad_code = partial(verif_code, pad=dirpad, avoid=(0, 0))

def generate_dirpad_steps(steps, dirpadpos, nr_of_dirpads, i):
    allres = []
    initial_pos = dirpadpos[i]
    for order in range(2):
        dirpadpos[i] = initial_pos
        res = [""]
        code = steps2code(steps, order)
        if not verif_dirpad_code(code, dirpadpos[i-1], i=i):
            continue
        ok = True
        for e in code:
            nsteps = steps_dirpad(dirpadpos[i], e)
            if i == nr_of_dirpads - 1:
                ncode = steps2code(nsteps, order)
                if not verif_dirpad_code(ncode, dirpadpos[i-1], i=i):
                    ok = False
                    break
                res = [e+ncode for e in res]
            else:
                subres = generate_dirpad_steps(nsteps, dirpadpos, nr_of_dirpads, i+1)
                res = [e1+e2 for e1 in res for e2 in subres]
            dirpadpos[i] = e
        if ok:
            allres.extend(res)
    return allres

def generate_steps(code, numpadpos="A", nr_of_dirpads=3):
    nr_of_dirpads -= 1
    dirpadpos = ["A"] * nr_of_dirpads
    res = [""]
    for e in code:
        steps = steps_numpad(numpadpos, e)
        if nr_of_dirpads:
            res = [e1+e2 for e1 in res for e2 in generate_dirpad_steps(steps, dirpadpos, nr_of_dirpads, 0)]
        else:
            res = [e+steps2code(steps) for e in res]
        numpadpos = e
    return res

In [ ]:
s = 0
for code in example[-1:]:
    allres = generate_steps(code, nr_of_dirpads=3)
    seq = sorted(allres, key=len)[0]
    print(code, seq, len(seq))
    s += len(seq)*int(code[:-1])
s